# Pandasによる主要な前処理

## 前提

ここでは効率的ににデータフレームを扱うために、処理を連続して行える方法を重視しています。

In [1]:
import pandas as pd
import numpy as np

## データの作成

In [2]:
data = pd.DataFrame({
    'cate1': ['a', 'a', 'a', 'b', 'b', 'c'],
    'cate2': ['A', 'B', 'B', 'C', 'C', 'C'],
    'value1': [1, 0, 1, 0, 1, 0],
    'value2': [10, 3, -4, -1, 0, 1]
})

In [3]:
data

,cate1,cate2,value1,value2
0,a,A,1,10
1,a,B,0,3
2,a,B,1,-4
3,b,C,0,-1
4,b,C,1,0
5,c,C,0,1


## 列の追加/演算

データ処理において最も基本と言って良い、演算や、その結果を新しい列として追加するなどの処理を行います。

In [4]:
target_data = data.assign(
    value_new=lambda x: x.value1 + x.value2
)
target_data[['value1', 'value2', 'value_new']]

,value1,value2,value_new
0,1,10,11
1,0,3,3
2,1,-4,-3
3,0,-1,-1
4,1,0,1
5,0,1,1


上の例では、新しい列名を 「value_new」 としているので、演算後の結果が新たな列になっていますが、既存の列名にするとその列の値が演算後の結果になります。

「assign」 は処理結果がデータフレームであるため、連続して処理を行えます。

In [5]:
target_data = data.assign(
    value_new=lambda x: x.value1 + x.value2
).assign(value_new=lambda x: x.value_new + 5)
target_data[['value1', 'value2', 'value_new']]

,value1,value2,value_new
0,1,10,16
1,0,3,8
2,1,-4,2
3,0,-1,4
4,1,0,6
5,0,1,6


### IFELSE

列の値を条件によって変えたいときに使うのがIFELSE処理です。

In [6]:
target_data = data.assign(
    value_new=lambda x: x.value2.map(lambda y: 1 if y > 0 else 0)
)
target_data[['value2', 'value_new']]

,value2,value_new
0,10,1
1,3,1
2,-4,0
3,-1,0
4,0,0
5,1,1


「assign」 の内部のlambdaで指定した 「x.value2」 はベクトル（Series）なので、ベクトル全体に対する処理は行なえますが、値一つ一つに対しての処理は行なえません。そこで、 「map」 を使って値一つ一つに対して処理を行っています。上の例では、値が0より大きい場合は１、それ以外の場合は０としています。

#### 複数列によるIFELSE

先程のやり方では、一つの列に対してIFELSE処理を行えますが、複数列を使った条件式のIFELSE処理は行なえません。これを行うために、 「apply」 を使います。

In [7]:
target_data['value_new'] = data.apply(
    lambda x: x['cate2'] if x['cate1'] == 'a' and x['value2'] > 0 else '◯',
    axis=1
)
target_data[['cate1', 'cate2', 'value2', 'value_new']]

,cate1,cate2,value2,value_new
0,a,A,10,A
1,a,B,3,B
2,a,B,-4,◯
3,b,C,-1,◯
4,b,C,0,◯
5,c,C,1,◯


#### 複数列によるIFELSE（高速化）

先程のやり方で複数列を条件としたIFELSE処理はできましたが、データ数が多くなったときに処理時間がかかります。この処理を高速化するためにnumpyを使います。

In [8]:
target_data['value_new'] = np.where(
    (data['cate1'].values == 'a') * (data['value2'].values > 0),
    data['cate2'].values,
    '◯'
)
target_data[['cate1', 'cate2', 'value2', 'value_new']]

,cate1,cate2,value2,value_new
0,a,A,10,A
1,a,B,3,B
2,a,B,-4,◯
3,b,C,-1,◯
4,b,C,0,◯
5,c,C,1,◯


データフレームで指定した列を 「values」 でnumpyに変換し、numpyのarray型（ベクトル）で条件式の処理をしています。AND式を行うためにBoolean型のベクトルの掛け算を行っています。また、OR式の場合は足し算です。

In [9]:
(data['cate1'].values == 'a') * (data['value2'].values > 0)

array([ True,  True, False, False, False, False])

##### 高速化の検証

In [10]:
%%timeit
target_data['value_new'] = data.apply(
    lambda x: x['cate2'] if x['cate1'] == 'a' and x['value2'] > 0 else '◯',
    axis=1
)

586 µs ± 21.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [11]:
%%timeit
target_data['value_new'] = np.where(
    (data['cate1'].values == 'a') * (data['value2'].values > 0),
    data['cate2'].values,
    '◯'
)

73.6 µs ± 2.99 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


平均値ではだいたい8倍ほど速くなっており、標準偏差も小さく安定しています。

## フィルター

指定した条件に当てはまる行を抽出します。

In [13]:
target_data = data.query('value1 > 0', engine='python')
target_data

,cate1,cate2,value1,value2
0,a,A,1,10
2,a,B,1,-4
4,b,C,1,0


In [14]:
target_data = data.query('cate1 == "a"', engine='python')
target_data

,cate1,cate2,value1,value2
0,a,A,1,10
1,a,B,0,3
2,a,B,1,-4


#### Nullを条件とした抽出

In [15]:
target_data = data.assign(
    value1=lambda x: x.value1.map(lambda y: np.NaN if y > 0 else y)
).query('value1.isnull()', engine='python')
target_data

,cate1,cate2,value1,value2
0,a,A,NaN,10
2,a,B,NaN,-4
4,b,C,NaN,0


In [16]:
target_data = data.assign(
    value1=lambda x: x.value1.map(lambda y: np.NaN if y > 0 else y)
).query('value1.notnull()', engine='python')
target_data

,cate1,cate2,value1,value2
1,a,B,0.0,3
3,b,C,0.0,-1
5,c,C,0.0,1


「isnull」 関数を使いNaNの行を抽出し、「notnull」 関数を使ってNaN以外の行を抽出しています。

「dropna」 関数を使ってもNaNを削除できます。

In [17]:
target_data = data.assign(
    value1=lambda x: x.value1.map(lambda y: np.NaN if y > 0 else y)
).dropna(subset=['value1'], axis=0)
target_data

,cate1,cate2,value1,value2
1,a,B,0.0,3
3,b,C,0.0,-1
5,c,C,0.0,1


#### 指定した文字を含む行を抽出

In [18]:
target_data = data.copy()
target_data['cate_new'] = target_data.apply(
    lambda x: x['cate1'] + x['cate2'], axis=1
)
target_data = target_data.query('cate_new.str.contains("C")', engine='python')
target_data

,cate1,cate2,value1,value2,cate_new
3,b,C,0,-1,bC
4,b,C,1,0,bC
5,c,C,0,1,cC


上の例では、文字列「C」を含む行を取り出しています。

## Grouping

統計演算を行う際に必ず出てくるのがGroupingです。これに慣れれば、どんな指標もサクッと作れます。

In [19]:
target_data = data.groupby(['cate1', 'cate2'])['value2'].mean().reset_index()
target_data

,cate1,cate2,value2
0,a,A,10.0
1,a,B,-0.5
2,b,C,-0.5
3,c,C,1.0


「cate1」 と 「cate2」 の２つの列を合わせた組み合わせが同じ行において、「value2」 の平均を算出しています。 「mean」 関数を変えれば、異なる集計ができます。

#### 同一列に複数演算

In [20]:
target_data = data.groupby(['cate1', 'cate2']).agg({
    'value2': ['count', 'mean', 'std']
}).reset_index()
target_data

cate1 cate2 value2                
               count  mean       std
0     a     A      1  10.0       NaN
1     a     B      2  -0.5  4.949747
2     b     C      2  -0.5  0.707107
3     c     C      1   1.0       NaN

「value2」 に対して、同時に複数の統計処理を行えます。

#### 自作関数

自作した関数を使ってのGroupingもできます。先程のGroupingでは、件数が１件の条件に関して、標準偏差がNaNになってしまいました。NaNの場合には０とする関数を作って、その関数を使ってGroupingします。

In [21]:
def std_fillna(x):
    return np.nan_to_num(np.std(x, ddof=1), 0)

In [22]:
target_data = data.groupby(['cate1', 'cate2']).agg({
    'value2': ['count', 'mean', std_fillna]
}).reset_index()
target_data

cate1 cate2 value2                 
               count  mean std_fillna
0     a     A      1  10.0   0.000000
1     a     B      2  -0.5   4.949747
2     b     C      2  -0.5   0.707107
3     c     C      1   1.0   0.000000

#### SummarizeせずにGrouping結果列に追加

上の例では、Groupingに指定した変数の重複でまとめらていますが、Groupingによる結果を新たな列として元のデータのままにしたい場合があります。

In [23]:
target_data = data.copy()
target_data['count'] = target_data.groupby(['cate1', 'cate2'])['value2'].transform('count')
target_data

,cate1,cate2,value1,value2,count
0,a,A,1,10,1
1,a,B,0,3,2
2,a,B,1,-4,2
3,b,C,0,-1,2
4,b,C,1,0,2
5,c,C,0,1,1


指定したGroupingの件数をデータのままの新しい列に残しています。

In [24]:
target_data = data.assign(number=1).copy()
target_data['number'] = target_data.groupby(['cate1', 'cate2'])['number'].transform('cumsum')
target_data

,cate1,cate2,value1,value2,number
0,a,A,1,10,1
1,a,B,0,3,1
2,a,B,1,-4,2
3,b,C,0,-1,1
4,b,C,1,0,2
5,c,C,0,1,1


指定したGroup単位で順番に番号を付与するなどもできます。

## 重複削除

指定した列で重複している値の行を消します。

In [25]:
target_data = data.drop_duplicates(['cate1', 'cate2'])[['cate1', 'cate2']]
target_data

,cate1,cate2
0,a,A
1,a,B
3,b,C
5,c,C


## ソート

指定した列の値によって、行を並び替えます。

In [26]:
target_data = data.sort_values(['value1', 'value2'], ascending=[True, False])
target_data

,cate1,cate2,value1,value2
1,a,B,0,3
5,c,C,0,1
3,b,C,0,-1
0,a,A,1,10
4,b,C,1,0
2,a,B,1,-4


上の例では、先に 「value1」 で昇順、次に 「value2」 で降順に並び替えています。「sort_values」 内の 「ascending」 で昇順か降順を指定しています。

ソートしてから重複削除もよく行います。

In [27]:
target_data = data.sort_values(
    ['value1', 'value2'], ascending=[True, False]
).drop_duplicates(['cate1', 'cate2'])
target_data

,cate1,cate2,value1,value2
1,a,B,0,3
5,c,C,0,1
3,b,C,0,-1
0,a,A,1,10


## 列名の変更

列名を変更します。

In [28]:
target_data = data.rename(columns={'cate1': 'cate_new', 'value1': 'value_new'})
target_data

,cate_new,cate2,value_new,value2
0,a,A,1,10
1,a,B,0,3
2,a,B,1,-4
3,b,C,0,-1
4,b,C,1,0
5,c,C,0,1


先程、Groupingしたときに列名がmulti_indexになってしまい、データフレームとしては扱いづらくなってしまいました。列名を書き換えることで、扱いやすい形に戻します。

In [29]:
target_data = data.groupby(['cate1', 'cate2']).agg({
    'value2': ['count', 'mean', 'std']
})
target_data.columns

MultiIndex([('value2', 'count'),
            ('value2',  'mean'),
            ('value2',   'std')],
           )

In [30]:
target_data.columns = list(map(
    lambda x: '{}_{}'.format(x[0], x[1]), target_data.columns
))
target_data.reset_index(inplace=True, drop=False)
target_data

,cate1,cate2,value2_count,value2_mean,value2_std
0,a,A,1,10.0,NaN
1,a,B,2,-0.5,4.949747
2,b,C,2,-0.5,0.707107
3,c,C,1,1.0,NaN


In [31]:
target_data.columns

Index(['cate1', 'cate2', 'value2_count', 'value2_mean', 'value2_std'], dtype='object')

## 値置換

値を置換します。よく使うのはNaNの置換です。

In [32]:
target_data = data.assign(
    value_new=lambda x: x.value2.map(lambda y: np.NaN if y >= 0 else y)
).assign(value_new=lambda x: x.value_new.fillna(0))
target_data[['value2', 'value_new']]

,value2,value_new
0,10,0.0
1,3,0.0
2,-4,-4.0
3,-1,-1.0
4,0,0.0
5,1,0.0


複数列の置換が必要なときは一気にやる場合もあります。

In [33]:
target_data = data.assign(
    value_new1=lambda x: x.value2.map(lambda y: np.NaN if y >= 0 else y),
    value_new2=lambda x: x.value2.map(lambda y: np.NaN if y < 0 else y),
)
target_data.fillna(0, inplace=True)
target_data[['value2', 'value_new1', 'value_new2']]

,value2,value_new1,value_new2
0,10,0.0,10.0
1,3,0.0,3.0
2,-4,-4.0,0.0
3,-1,-1.0,0.0
4,0,0.0,0.0
5,1,0.0,1.0


## 結合

異なるデータを列の値を基準としてつなぎ合わせたいときに結合処理を行います。

In [34]:
target_data = pd.merge(
    data[['cate1', 'cate2', 'value1']],
    data.groupby('cate1')['value1'].count().reset_index().rename(
        columns={'value1': 'count'}
    ),
    on='cate1', how='inner'
)
target_data

,cate1,cate2,value1,count
0,a,A,1,3
1,a,B,0,3
2,a,B,1,3
3,b,C,0,2
4,b,C,1,2
5,c,C,0,1


In [35]:
target_data = pd.merge(
    data[['cate1', 'cate2', 'value1']],
    data.query('cate1 == "a"').assign(flag=1)[['cate1', 'flag']].drop_duplicates('cate1'),
    on='cate1', how='left'
)
target_data

,cate1,cate2,value1,flag
0,a,A,1,1.0
1,a,B,0,1.0
2,a,B,1,1.0
3,b,C,0,NaN
4,b,C,1,NaN
5,c,C,0,NaN


### semi_join

「semi_join」 は、結合されるデータから行を抽出する際に、結合するデータの指定した列の値が同じ行だけを抽出する処理です。<br>
R言語のdplyrにはsemi_joinがありますが、pandasにはありません（たぶん）。以下のように自作しました。

In [36]:
def semi_join(data1, data2, by):

    if isinstance(by, str):
        by = [by]

    return pd.merge(data2[by].drop_duplicates(), data1, how='inner', on=by)

In [37]:
target_data = semi_join(
    data,
    data.query('cate1 == "a"'),
    by='cate1'
)
target_data

,cate1,cate2,value1,value2
0,a,A,1,10
1,a,B,0,3
2,a,B,1,-4


#### anti_join

In [38]:
def anti_join(data1, data2, by):

    joined_data = data1.copy()
    target_data = data2.copy()
    target_data['flag_tmp'] = 1

    if isinstance(by, str):
        by = [by]

    joined_data = pd.merge(
        joined_data, target_data[by + ['flag_tmp']].drop_duplicates(),
        on=by, how='left'
    ).query('flag_tmp.isnull()', engine='python').drop(
        columns='flag_tmp'
    ).reset_index(drop=True)

    return joined_data

「anti_join」 は、結合されるデータから行を抽出する際に、結合するデータの指定した列の値が異なる行だけを抽出する処理です。

In [39]:
target_data = anti_join(
    data,
    data.query('cate1 == "a"'),
    by='cate1'
)
target_data

,cate1,cate2,value1,value2
0,b,C,0,-1
1,b,C,1,0
2,c,C,0,1


## 連結

### 縦の結合

In [40]:
target_data = pd.concat([data, data], axis=0).reset_index(drop=True)
target_data

,cate1,cate2,value1,value2
0,a,A,1,10
1,a,B,0,3
2,a,B,1,-4
3,b,C,0,-1
4,b,C,1,0
5,c,C,0,1
6,a,A,1,10
7,a,B,0,3
8,a,B,1,-4
9,b,C,0,-1


### 横の結合

In [41]:
target_data = pd.concat([
    data,
    data.rename(columns={'cate1': 'cate_new', 'value1': 'value_new'})[['cate_new', 'value_new']]
], axis=1)
target_data

,cate1,cate2,value1,value2,cate_new,value_new
0,a,A,1,10,a,1
1,a,B,0,3,a,0
2,a,B,1,-4,a,1
3,b,C,0,-1,b,0
4,b,C,1,0,b,1
5,c,C,0,1,c,0


## 縦横変形

グラフで扱いやすいデータにするために、複数の列を一つの列にまとめる縦変形や、機械学習モデルに読み込ませるための横変形などがあり、これはよく行います。

#### 縦変形

In [42]:
target_data = data.assign(id=1).assign(
    id=lambda x: x.id.cumsum()
).set_index(['id', 'cate1', 'cate2']).stack().reset_index()
target_data.columns = ['id', 'cate1', 'cate2', 'variable', 'value']
target_data

,id,cate1,cate2,variable,value
0,1,a,A,value1,1
1,1,a,A,value2,10
2,2,a,B,value1,0
3,2,a,B,value2,3
4,3,a,B,value1,1
5,3,a,B,value2,-4
6,4,b,C,value1,0
7,4,b,C,value2,-1
8,5,b,C,value1,1
9,5,b,C,value2,0


これにより「seaborn」でのグラフ化が行いやすくなります。

### 横変形

In [43]:
stacked_data = data.assign(id=1).assign(
    id=lambda x: x.id.cumsum()
).set_index(['id', 'cate1', 'cate2']).stack().reset_index()
stacked_data.columns = ['id', 'cate1', 'cate2', 'variable', 'value']

target_data = pd.pivot_table(
    data=stacked_data,
    index=['id', 'cate1', 'cate2'],
    columns='variable'
)
target_data.columns = list(map(lambda x: x[1], target_data.columns))
target_data.reset_index(inplace=True, drop=False)
target_data

,id,cate1,cate2,value1,value2
0,1,a,A,1,10
1,2,a,B,0,3
2,3,a,B,1,-4
3,4,b,C,0,-1
4,5,b,C,1,0
5,6,c,C,0,1


##### 文字列を値とする横変形

「pivot_table」 は横変形するときに値を平均化するのがデフォルトの挙動です。なので、文字列を値とするとエラーになります。

In [44]:
stacked_data = data.assign(id=1).assign(
    id=lambda x: x.id.cumsum()
).set_index(['id', 'cate1', 'cate2']).stack().reset_index()
stacked_data.columns = ['id', 'cate1', 'cate2', 'variable', 'value']

target_data = pd.pivot_table(
    data=stacked_data[['id', 'cate1', 'cate2']].drop_duplicates(),
    index='id',
    columns='cate1',
    fill_value=''
)
target_data.columns = list(map(lambda x: x[1], target_data.columns))
target_data.reset_index(inplace=True, drop=False)
target_data

DataError: No numeric types to aggregate

これは、「pivot_table」 内のagg_func引数のデフォルトが 「np.mean」 となっているからであり、この引数を変えることで文字列を値とすることができます。

In [45]:
stacked_data = data.assign(id=1).assign(
    id=lambda x: x.id.cumsum()
).set_index(['id', 'cate1', 'cate2']).stack().reset_index()
stacked_data.columns = ['id', 'cate1', 'cate2', 'variable', 'value']

target_data = pd.pivot_table(
    data=stacked_data[['id', 'cate1', 'cate2']].drop_duplicates(),
    index='id',
    columns='cate1',
    fill_value='',
    aggfunc=lambda x: x
)
target_data.columns = list(map(lambda x: x[1], target_data.columns))
target_data.reset_index(inplace=True, drop=False)
target_data

,id,a,b,c
0,1,A,,
1,2,B,,
2,3,B,,
3,4,,C,
4,5,,C,
5,6,,,C


上記の例では、「agg_func」 引数に 「lambda x: x」 を代入しており、値をそのまま返すようにしています。